# ANALIZA PODATKOV O HOTELIH

V tem delu bomo pridobljene podatke predelali, analizirali in jih primerjali.

In [2]:
# tabele bomo analizirali s pomočjo pandas, zato ga naložimo
import pandas as pd

# ker so razpredelnice velike, povemo, naj vedno izpiše le prvih 10 vrstic
pd.set_option('display.max_rows', 10)

# naložimo csv-je
amsterdam = pd.read_csv('amsterdam.csv')
split = pd.read_csv('split.csv')
lizbona = pd.read_csv('lizbona.csv') 

Izpišimo tabele, da si bomo lažje predstavljali podatke, s akterimi imamo opravka.

In [51]:
amsterdam

,lokacija,ime,cena,ocena gostov,število zvezdic
0,Amsterdam,Orange Tulip Hotel,€ 328,"6,2",1
1,Amsterdam,Hotel Continental Amsterdam,€ 508,"5,1",1
2,Amsterdam,Amsterdam Downtown Hotel,€ 419,"6,9",2
3,Amsterdam,Weber Hotel,€ 547,"8,1",2
4,Amsterdam,Hotel Manofa,€ 333,"7,2",2
...,...,...,...,...,...
138,Amsterdam,Andaz Amsterdam Prinsengracht - a concept by H...,€ 1.007,"8,8",5
139,Amsterdam,Amsterdam Marriott Hotel,€ 872,"8,0",5
140,Amsterdam,Anantara Grand Hotel Krasnapolsky Amsterdam,€ 842,"8,7",5
141,Amsterdam,De L’Europe Amsterdam – The Leading Hotels of ...,€ 1.169,"8,9",5


In [38]:
split

,lokacija,ime,cena,ocena gostov,število zvezdic
0,Split,Hotel Alem,€ 119,"7,1",2
1,Split,Fontana Resort,€ 45,"7,0",2
2,Split,Arkada Hotel,€ 127,"6,9",2
3,Split,Hotel Zagreb,€ 114,"6,9",2
4,Split,Slavija Culture Heritage Hotel,€ 380,"8,7",3
...,...,...,...,...,...
156,Split,Damianii Luxury Boutique Hotel & Spa,€ 304,"8,8",5
157,Split,Bluesun hotel Berulia,€ 281,"8,9",5
158,Split,Bluesun Hotel Jadran,€ 294,"9,3",5
159,Split,Heritage Hotel Martinis Marchi,€ 418,"9,6",5


In [39]:
lizbona

,lokacija,ime,cena,ocena gostov,število zvezdic
0,Lizbona,Patria Hotel,€ 178,"8,1",1
1,Lizbona,Hotel Nova Cidade,€ 80,"7,9",1
2,Lizbona,Stay Hotel Lisboa Centro Saldanha,€ 131,"7,7",2
3,Lizbona,Moov Hotel Lisboa Oriente,€ 153,"8,6",2
4,Lizbona,Hotel Gat Rossio,€ 184,"8,5",2
...,...,...,...,...,...
192,Lizbona,VIP Grand Lisboa Hotel & Spa,€ 243,"7,7",5
193,Lizbona,Corpo Santo Lisbon Historical Hotel,€ 560,"9,4",5
194,Lizbona,Four Seasons Hotel Ritz Lisbon,€ 1.338,"8,9",5
195,Lizbona,Hotel Valverde Lisboa - Relais & Chateaux,€ 613,"8,8",5


Kot pričakovano, nas najprej zanima katera lokacija ima v povprečju najdražje oz. najcenejše prenočitve in katera v poprečju najvišje oz. najnižje ocene.

In [53]:
# Da bomo lažje analizirati združimo vse tabele v eno
hoteli = pd.concat([amsterdam, split, lizbona], ignore_index=True)



# Cene hotelov so napisane kot nizi oblike '€ 123', oz. '€ 1.023', če so dražje kot 999€. Da bomo z njimi lahko računali povprečja, 
# jih moramo zato preurediti v samo številske znake in jih narediti v numerični tip.
hoteli['cena'] = hoteli['cena'].replace({'€': '', ' ': ''}, regex=True).str.replace('.', '', regex=False) # uporabimo regex=False, da ne bi python razmel pike kot 'katerikoli znak'
hoteli['cena'] = pd.to_numeric(hoteli['cena'])

# Enako moramo storiti z ocenami, ki so napisani kot nizi oblike '8,7'. Python za decimalko uporablja piko, zato moramo popraviti vejico in
# niz pretvoriti v numerični tip.
hoteli['ocena gostov'] = hoteli['ocena gostov'].replace({',': '.'}, regex=True)
hoteli['ocena gostov'] = pd.to_numeric(hoteli['ocena gostov'])



# Izračunamo povprečne cene in ocene gostov po lokacijah
povprecja = hoteli.groupby('lokacija').agg({'cena': 'mean', 'ocena gostov': 'mean'}).reset_index()
povprecja['cena'] = povprecja['cena'].round(2) # zaokrožimo na eno decimalko
povprecja['ocena gostov'] = povprecja['ocena gostov'].round(1) # zaokrožimo na eno decimalko

# Preimenujemo stolpce za boljšo preglednost
povprecja.rename(columns={'cena': 'povprečna cena', 'ocena gostov': 'povprečna ocena'}, inplace=True)

print(povprecja) # izpišemo tabelo s povprečji



# V primeru da je lokacij preveč, je uporabno da nam program izpiše katera ima najdražje/najcenejše prenočitve in najboljše/najslabše ocene
najdrazja_lokacija = povprecja.loc[povprecja['povprečna cena'].idxmax()]['lokacija']
najcenejsa_lokacija = povprecja.loc[povprecja['povprečna cena'].idxmin()]['lokacija']

najboljsa_lokacija = povprecja.loc[povprecja['povprečna ocena'].idxmax()]['lokacija']
najslabsa_lokacija = povprecja.loc[povprecja['povprečna ocena'].idxmin()]['lokacija']


print()
print(f"V povprečju ima najdražje prenočitve {najdrazja_lokacija} in najcenejše prenočitve {najcenejsa_lokacija}.")
print(f"V povprečju ima najboljše ocenjene hotele {najboljsa_lokacija} in najslabše ocenjene hotele {najslabsa_lokacija}.")

    lokacija  povprečna cena  povprečna ocena
0  Amsterdam          521.28              8.0
1    Lizbona          303.98              8.5
2      Split          216.42              8.7

V povprečju ima najdražje prenočitve Amsterdam in najcenejše prenočitve Split.
V povprečju ima najboljše ocenjene hotele Split in najslabše ocenjene hotele Amsterdam.
